# Here i want to extract from lichess data features important for future modeling

So here i want to get data in format similar to that: 

columns = ( date, amount of mathes played in that date,ammont of matches played day earllier , amount of matches plyed for previous mounth, ranikng points fluctuation (day vs day before), ranking after last match that day, mean oponent vs my ranking diff that day ) and maybe some other columns that i will consider as important for ranking changes prediction. --> df2

And all of those for different times control (blitz, rapid, bullet)

and other data for winner prediction models, similar columns as without any chnages but some new cols like: time used in game, time for move and maybe some other --> df1

# To Be Continued

In [105]:
import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import plot_tree

from sklearn.model_selection import train_test_split
# Regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
# Clasification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, accuracy_score
# Best_searches
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
# Standarization
from sklearn.preprocessing import StandardScaler

from keras import models
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score, precision_score, accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import numpy as np

In [106]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import pandas as pd
import numpy as np
import arrow
pd.set_option('display.max_columns', None)

In [41]:
df = pd.read_csv("Chess_data_basic.csv", index_col = "Unnamed: 0")

In [42]:
len(set(df["black_player_name"]+df["white_player_name"]))

4789

In [43]:
df.isna().sum()

id                     0
rated                  0
variant                0
speed                  0
createdAt              0
lastMoveAt             0
status                 0
winner               250
moves                  1
clock                  0
movesperplayer         0
white_player_name      0
white_ranking          0
black_player_name      0
black_ranking          0
dtype: int64

In [44]:
def convert_date_with_offset(date_str):
    return arrow.get(date_str).datetime

for i in ['createdAt', 'lastMoveAt']:
    df[i] = df[i].apply(convert_date_with_offset)
    
# Create Gametime column
df["Gametime"] = df["lastMoveAt"] - df["createdAt"]
# Handle Gametime column
df['Gametime'] = pd.to_timedelta(df['Gametime'])

rounding = lambda x: round(x, 1)

df['Gametime'] = df['Gametime'].dt.total_seconds().apply(rounding)
df['timepermove']=(df['Gametime']/df["movesperplayer"]).apply(rounding)

next think to do is taking insight from moves column, there is a lot of possibility but i will concentrate on:
1) cheking if side castle and if long or short castle
2) taking number of moves before castle ( if no castle - values set to number of moves +1 )
3) groping by opening (for this project i will take in count only first 6 moves - 3 moves per side )
4) number of checks in first 10 moves ( per side )

## Handle NA values

In [45]:
# Only timepermove colum had NA values (just one), expect winner, where NA == DRAW
df.dropna(subset=['timepermove'],inplace=True)

df = df.fillna("draw")

df.isna().sum()

id                   0
rated                0
variant              0
speed                0
createdAt            0
lastMoveAt           0
status               0
winner               0
moves                0
clock                0
movesperplayer       0
white_player_name    0
white_ranking        0
black_player_name    0
black_ranking        0
Gametime             0
timepermove          0
dtype: int64

# Checking for side castle 

kingside castle ( short ) is O-O in moves notation and queenside castle ( long ) is O-O-O

In [46]:
pd.options.mode.chained_assignment = None

In [47]:
def moves_column(df):
    df["moves"] = df["moves"].str.split(" ")
    df["white_moves"]=df["moves"].apply(lambda x: x[::2])
    df["black_moves"]=df["moves"].apply(lambda x: x[1::2])
    return df

df = moves_column(df)

In [48]:
def castle( df ):
    color_list = []
    my_short = []
    my_long = []
    enemy_short = []
    enemy_long = []
    
    
    for index, row in df.iterrows():
        if df["white_player_name"][index]=="EvilSaintPL":
            color_list.append(1)
        else:
            color_list.append(0)
        
        if df["white_player_name"][index]=="EvilSaintPL" and ("O-O" in df["white_moves"][index]):
            my_short.append(1)
            my_long.append(0)
        elif df["white_player_name"][index]=="EvilSaintPL" and ("O-O-O" in df["white_moves"][index]):
            my_long.append(1)
            my_short.append(0)
        elif df["black_player_name"][index]=="EvilSaintPL" and ("O-O" in df["black_moves"][index]):
            my_short.append(1)
            my_long.append(0)
        elif df["black_player_name"][index]=="EvilSaintPL" and ("O-O-O" in df["black_moves"][index]):
            my_long.append(1)
            my_short.append(0)
        else:
            my_long.append(0)
            my_short.append(0)
        
        if df["white_player_name"][index]!="EvilSaintPL" and ("O-O" in df["white_moves"][index]):
            enemy_short.append(1)
            enemy_long.append(0)
        elif df["white_player_name"][index]!="EvilSaintPL" and ("O-O-O" in df["white_moves"][index]):
            enemy_long.append(1)
            enemy_short.append(0)
        elif df["black_player_name"][index]!="EvilSaintPL" and ("O-O" in df["black_moves"][index]):
            enemy_short.append(1)
            enemy_long.append(0)
        elif df["black_player_name"][index]!="EvilSaintPL" and ("O-O-O" in df["black_moves"][index]):
            enemy_long.append(1)
            enemy_short.append(0)
        else:
            enemy_long.append(0)
            enemy_short.append(0)        
        
    df["my_kingcastle"]=my_short
    df["my_queencastle"]=my_long
    df["enemy_kingcastle"]=enemy_short
    df["enemy_queencastle"]=enemy_long
    df["my_color"]=color_list
    return df
        
df = castle(df)

# num of moves before castle 

In [49]:
def find_index_of_first_occurrence(lst):
    elements = ["O-O", "O-O-O"]
    for element in elements:
        if element in lst:
            return lst.index(element)
    return len(lst) + 1


In [50]:
def moves_before_castle( df ):
    df["White_moves_before_castle"] = df["white_moves"].apply(find_index_of_first_occurrence)
    df["Black_moves_before_castle"] = df["black_moves"].apply(find_index_of_first_occurrence)
    
    my_moves_before = []
    enemy_moves_before = []
    
    for index, row in df.iterrows():
        if df["my_color"][index]==1:
            my_moves_before.append(df["White_moves_before_castle"][index])
            enemy_moves_before.append(df["Black_moves_before_castle"][index])
        else:
            my_moves_before.append(df["Black_moves_before_castle"][index])
            enemy_moves_before.append(df["White_moves_before_castle"][index])
            
    df["my_moves_before_castle"] = my_moves_before
    df["enemy_moves_before_castle"]= enemy_moves_before
    
    for index, row in df.iterrows():
        if df["my_moves_before_castle"][index]>(df["movesperplayer"][index]):
            df["my_moves_before_castle"][index]=100
        elif df["enemy_moves_before_castle"][index]>(df["movesperplayer"][index]):
            df["enemy_moves_before_castle"][index]=100
    
    df["my_moves_before_castle"] = df["my_moves_before_castle"]/(100)
    df["enemy_moves_before_castle"] = df["enemy_moves_before_castle"]/(100)
    
    return df

df=moves_before_castle( df )

## Grouping by opening

In [51]:
df["open4"] = df["moves"].apply(lambda x: x[:4])
df["open4"] = df["open4"].apply(lambda x: " ".join(x))

In [52]:
white_opens = df[df['white_player_name'] == 'EvilSaintPL']["open4"].value_counts().head(20)
black_opens = df[df['black_player_name'] == 'EvilSaintPL']["open4"].value_counts().head(20)

In [53]:
def iterable_opening_list_and_my_ranking( df ):
    
    openings_list = []
    rank_of_evilsaintpl_list = []
    evilsaintpl_won = []
    enemy_rank_list = []

    
    for index, row in df.iterrows():
        
        # OPENS1
        if df["white_player_name"][index]=="EvilSaintPL" and df["open4"][index] in white_opens.index:
            openings_list.append("White " + df["open4"][index])
        elif  df["white_player_name"][index]!="EvilSaintPL" and df["open4"][index] in black_opens.index:
            openings_list.append("Black " + df["open4"][index]) 
        else:
            openings_list.append("no opens")
        
        # MY RANK AND ENEMY RANK 
        if df["white_player_name"][index]=="EvilSaintPL":
            rank_of_evilsaintpl_list.append(df["white_ranking"][index])
            enemy_rank_list.append(df["black_ranking"][index])
        elif df["black_player_name"][index]=="EvilSaintPL":
            rank_of_evilsaintpl_list.append(df["black_ranking"][index])
            enemy_rank_list.append(df["white_ranking"][index])
        else:
            print(index)
            
        # I WON 
        if df["winner"][index] == 'draw':
            evilsaintpl_won.append(0)
        elif df["white_player_name"][index]=="EvilSaintPL" and df["winner"][index]=='white':
            evilsaintpl_won.append(1)
        elif  df["black_player_name"][index]=="EvilSaintPL" and df["winner"][index]=='black':
            evilsaintpl_won.append(1)
        else:
            evilsaintpl_won.append(-1)

    df["opens1"]= openings_list
    df["my_rank"]= rank_of_evilsaintpl_list
    df['i_won'] = evilsaintpl_won
    df['enemy_rank'] = enemy_rank_list
    
    # NEW RANKS FE
    df['rank_difference'] = df['my_rank'] - df['enemy_rank']
    df['is_higher'] = (df['my_rank'] > df['enemy_rank'])


    return df

df = iterable_opening_list_and_my_ranking(df)

In [54]:
df.head(1)

,id,rated,variant,speed,createdAt,lastMoveAt,status,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,my_kingcastle,my_queencastle,enemy_kingcastle,enemy_queencastle,my_color,White_moves_before_castle,Black_moves_before_castle,my_moves_before_castle,enemy_moves_before_castle,open4,opens1,my_rank,i_won,enemy_rank,rank_difference,is_higher
0,O83cQxfo,True,standard,blitz,2023-06-28 02:33:43.673000+02:00,2023-06-28 02:44:00.722000+02:00,outoftime,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,1,6,71,0.06,1.0,e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nc6,1753,1,1735,18,True


In [55]:
def clear_dataframe(df):
    temp = pd.get_dummies(df["opens1"])
    df = pd.concat([df,temp], axis=1)
    
    df.drop("status",axis=1, inplace=True)
    df.drop(labels = ["opens1"], axis=1, inplace=True)
#    df.drop("id",axis=1,inplace=True)
    
    df = df[df["rated"]==True]
    df = df[df["white_player_name"]!="YoungCapitan"]
    df = df[df["black_player_name"]!="YoungCapitan"]
    df.drop("variant",axis=1,inplace=True)
    
    return df

df = clear_dataframe(df)

In [56]:
df.head(1)

,id,rated,speed,createdAt,lastMoveAt,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,my_kingcastle,my_queencastle,enemy_kingcastle,enemy_queencastle,my_color,White_moves_before_castle,Black_moves_before_castle,my_moves_before_castle,enemy_moves_before_castle,open4,my_rank,i_won,enemy_rank,rank_difference,is_higher,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7,no opens
0,O83cQxfo,True,blitz,2023-06-28 02:33:43.673000+02:00,2023-06-28 02:44:00.722000+02:00,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,1,6,71,0.06,1.0,e4 e5 Nf3 Nc6,1753,1,1735,18,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [57]:
df["i_won"]=df["i_won"]+1

In [58]:
df.shape

(4880, 73)

In [59]:
df.speed.unique()

array(['blitz', 'classical', 'bullet', 'rapid'], dtype=object)

In [60]:
def rankdifftest(df):
    
    blitz = df[df["speed"]=="blitz"].reset_index(drop=True)
    classical = df[df["speed"]=="classical"].reset_index(drop=True)
    bullet = df[df["speed"]=="bullet"].reset_index(drop=True)
    rapid = df[df["speed"]=="rapid"].reset_index(drop=True)
    
    for speed in [blitz, classical, bullet, rapid]:
        rank_resu=list()
        for index, row in speed.iterrows():
            if index==0:
                rank_resu.append(speed["my_rank"][index])
            else:
                rank_resu.append(speed["my_rank"][index-1])
        speed["rank_after_game"]=rank_resu
        speed["my_rank_diff"]=speed["rank_after_game"]-speed["my_rank"]
    return pd.concat([blitz, classical, bullet, rapid])

In [61]:
df = rankdifftest(df)

In [62]:
df.shape

(4880, 75)

In [63]:
df.head(2)

,id,rated,speed,createdAt,lastMoveAt,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,my_kingcastle,my_queencastle,enemy_kingcastle,enemy_queencastle,my_color,White_moves_before_castle,Black_moves_before_castle,my_moves_before_castle,enemy_moves_before_castle,open4,my_rank,i_won,enemy_rank,rank_difference,is_higher,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7,no opens,rank_after_game,my_rank_diff
0,O83cQxfo,True,blitz,2023-06-28 02:33:43.673000+02:00,2023-06-28 02:44:00.722000+02:00,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,1,6,71,0.06,1.00,e4 e5 Nf3 Nc6,1753,2,1735,18,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,1753,0
1,KeBcqcHz,True,blitz,2023-06-28 02:31:45.081000+02:00,2023-06-28 02:33:30.178000+02:00,white,"[Nf3, Nc6, g3, e5, Bg2, e4, Ng1, d5, d3, f5, d...",300,18,convict-19,1801,EvilSaintPL,1758,105.1,5.8,"[Nf3, g3, Bg2, Ng1, d3, dxe4, Nc3, Bg5, Bxf6, ...","[Nc6, e5, e4, d5, f5, fxe4, Nf6, Be7, Bxf6, Qx...",0,1,1,0,0,17,14,0.14,0.17,Nf3 Nc6 g3 e5,1758,0,1801,-43,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,1753,-5


In [64]:
model_columns = [ 'i_won','my_color','my_rank', 'enemy_rank','rank_difference', 'is_higher', 'movesperplayer', 'Gametime', 'timepermove','my_kingcastle', 'my_queencastle',
       'enemy_kingcastle', 'enemy_queencastle',
       'my_moves_before_castle', 'enemy_moves_before_castle', 'no opens', 'Black c4 e5 Nc3 Nf6',
       'Black d4 Nf6 Bf4 e6', 'Black d4 Nf6 Nf3 e6', 'Black d4 Nf6 c4 e6',
       'Black d4 d5 Bf4 Nf6', 'Black d4 d5 c4 c6', 'Black e4 e5 Bc4 Nc6',
       'Black e4 e5 Bc4 Nf6', 'Black e4 e5 Nc3 Nf6', 'Black e4 e5 Nf3 Nc6',
       'Black e4 e5 Nf3 Nf6', 'Black e4 e5 d4 exd4', 'Black e4 e5 f4 d5',
       'Black e4 e5 f4 exf4', 'Black e4 e6 Bc4 d5', 'Black e4 e6 Nc3 d5',
       'Black e4 e6 Nf3 d5', 'Black e4 e6 d4 d5', 'Black e4 e6 e5 c5',
       'Black g3 e5 Bg2 Nf6', 'White e4 c5 Nf3 Nc6', 'White e4 c5 Nf3 d6',
       'White e4 c5 Nf3 e6', 'White e4 c6 Nf3 d5', 'White e4 c6 d4 d5',
       'White e4 d5 Nf3 dxe4', 'White e4 d5 exd5 Nf6', 'White e4 d5 exd5 Qxd5',
       'White e4 e5 Nc3 Nc6', 'White e4 e5 Nc3 Nf6', 'White e4 e5 Nc3 d6',
       'White e4 e5 Nf3 Bc5', 'White e4 e5 Nf3 Nc6', 'White e4 e5 Nf3 Nf6',
       'White e4 e5 Nf3 Qf6', 'White e4 e5 Nf3 d5', 'White e4 e5 Nf3 d6',
       'White e4 e6 Nf3 d5', 'White e4 e6 d4 d5', 'White e4 g6 d4 Bg7'  ]

In [65]:
final_all = df[model_columns]

blitz = df[df["speed"]=="blitz"][model_columns]

classical = df[df["speed"]=="classical"][model_columns]

bullet = df[df["speed"]=="bullet"][model_columns]

rapid = df[df["speed"]=="rapid"][model_columns]

In [66]:
print(f"blitz : {blitz.shape},\nclassic: {classical.shape},\nbullet: {bullet.shape},\nrapid: {rapid.shape},\nfull: {final_all.shape}")

blitz : (3417, 56),
classic: (153, 56),
bullet: (975, 56),
rapid: (335, 56),
full: (4880, 56)


In [67]:
blitz.head(2)

,i_won,my_color,my_rank,enemy_rank,rank_difference,is_higher,movesperplayer,Gametime,timepermove,my_kingcastle,my_queencastle,enemy_kingcastle,enemy_queencastle,my_moves_before_castle,enemy_moves_before_castle,no opens,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7
0,2,1,1753,1735,18,True,70,617.0,8.8,1,0,0,0,0.06,1.00,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
1,0,0,1758,1801,-43,False,18,105.1,5.8,0,1,1,0,0.14,0.17,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [68]:
rapid.head(2)

,i_won,my_color,my_rank,enemy_rank,rank_difference,is_higher,movesperplayer,Gametime,timepermove,my_kingcastle,my_queencastle,enemy_kingcastle,enemy_queencastle,my_moves_before_castle,enemy_moves_before_castle,no opens,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7
0,0,1,1790,1785,5,True,62,1079.0,17.4,1,0,1,0,0.10,0.09,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
1,2,0,1777,1783,-6,False,37,845.4,22.8,1,0,0,1,0.08,0.10,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [69]:
df.speed.unique()

array(['blitz', 'classical', 'bullet', 'rapid'], dtype=object)

In [70]:
game_modes = [blitz, classical, bullet, rapid, final_all]
game_modes_fixed = []

def fix_infinity():
    for i in game_modes:
        i = i[~i["timepermove"].isin([np.inf, -np.inf])]
        game_modes_fixed.append(i)
    return game_modes_fixed

blitz, classical, bullet, rapid, final_all = fix_infinity()

In [71]:
# blitz = blitz[~blitz["timepermove"].isin([np.inf, -np.inf])]

In [72]:
bullet[bullet['timepermove'] > 1000]

,i_won,my_color,my_rank,enemy_rank,rank_difference,is_higher,movesperplayer,Gametime,timepermove,my_kingcastle,my_queencastle,enemy_kingcastle,enemy_queencastle,my_moves_before_castle,enemy_moves_before_castle,no opens,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7


In [73]:
# column definiton
"""
i_won - 0 -> I lost, 1-Draw, 2-I_Won
is_higher -> is my rank higher than my enemy
timepermove -> Gametime / movesperplayer
White_moves_before_castle -> amount of white moves before castle
no opens - no specyfic opening typed played by evilsainpl
"""

'\ni_won - 0 -> I lost, 1-Draw, 2-I_Won\nis_higher -> is my rank higher than my enemy\ntimepermove -> Gametime / movesperplayer\nWhite_moves_before_castle -> amount of white moves before castle\nno opens - no specyfic opening typed played by evilsainpl\n'

In [74]:
X = final_all.drop("i_won",axis=1)
y = final_all['i_won']

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [78]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn import tree, ensemble, naive_bayes, neighbors, semi_supervised, discriminant_analysis, svm, linear_model, neural_network


RandomForest_params = {'max_depth': 5,
 'min_samples_leaf': 2,
 'min_samples_split': 10,
 'n_estimators': 300}
RF = ensemble.RandomForestClassifier(**RandomForest_params)
RF.fit(X_train, y_train)




RandomForestClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=10,
                       n_estimators=300)

In [79]:
X.dtypes

my_color                       int64
my_rank                        int64
enemy_rank                     int64
rank_difference                int64
is_higher                       bool
movesperplayer                 int64
Gametime                     float64
timepermove                  float64
my_kingcastle                  int64
my_queencastle                 int64
enemy_kingcastle               int64
enemy_queencastle              int64
my_moves_before_castle       float64
enemy_moves_before_castle    float64
no opens                        bool
Black c4 e5 Nc3 Nf6             bool
Black d4 Nf6 Bf4 e6             bool
Black d4 Nf6 Nf3 e6             bool
Black d4 Nf6 c4 e6              bool
Black d4 d5 Bf4 Nf6             bool
Black d4 d5 c4 c6               bool
Black e4 e5 Bc4 Nc6             bool
Black e4 e5 Bc4 Nf6             bool
Black e4 e5 Nc3 Nf6             bool
Black e4 e5 Nf3 Nc6             bool
Black e4 e5 Nf3 Nf6             bool
Black e4 e5 d4 exd4             bool
B

In [89]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score, precision_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier



In [116]:
def model_for_different_gamemodes():
    final_results = {}
    all_dataframes = [blitz, classical, bullet, rapid, final_all]
    dataframes_names_str = ["blitz", "classical", "bullet", "rapid", "all_data"]
    
    for dataframe, dataframe_name in zip(all_dataframes, dataframes_names_str):
        
        X = dataframe.drop("i_won", axis=1)
        y = dataframe['i_won']
        # Define the parameter grid for grid search
        
        scaler = MinMaxScaler()
        X_normalized = scaler.fit_transform(X)

        X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, random_state = 0)  
        
        param_dist = {
            'n_estimators': np.arange(100, 500, 100),
            'max_depth': np.arange(1, 70, 7),
            'min_samples_leaf': np.arange(2, 70, 7),
            'min_samples_split': np.arange(2, 70, 7),
            'max_features': ['sqrt', 'log2']
        }

        # Create the Random Forest classifier
        rfc = RandomForestClassifier()

        # Create the randomized search object
        random_search = RandomizedSearchCV(rfc, param_distributions=param_dist, cv=5, n_iter=10, error_score='raise')

        # Perform randomized search on the training data
        random_search.fit(X_train, y_train)

        # Get the best estimator and its corresponding parameters
        best_rf = random_search.best_estimator_
        print('Best Params: ', random_search.best_params_)

        # Fit the best estimator on the training data
        best_rf.fit(X_train, y_train)

        # Predict on the training and test data using the best estimator
        y_pred_train = best_rf.predict(X_train)
        y_pred = best_rf.predict(X_test)
                
        final_results[dataframe_name] = {}
        # Calculate the evaluation metrics
        final_results[dataframe_name]["Trainging Set metics"] = {}
        final_results[dataframe_name]["Trainging Set metics"]['f1_score'] = f1_score(y_train, y_pred_train, average='weighted', zero_division=1)
        final_results[dataframe_name]["Trainging Set metics"]['precision_score'] = precision_score(y_train, y_pred_train, average='weighted', zero_division=1)
        final_results[dataframe_name]["Trainging Set metics"]['accuracy_score'] = accuracy_score(y_train, y_pred_train)
        
        final_results[dataframe_name]["Test Set metics"] = {}
        final_results[dataframe_name]["Test Set metics"]['f1_score'] = f1_score(y_test, y_pred, average='weighted', zero_division=1)
        final_results[dataframe_name]["Test Set metics"]['precision_score'] = precision_score(y_test, y_pred, average='weighted', zero_division=1)
        final_results[dataframe_name]["Test Set metics"]['accuracy_score'] = accuracy_score(y_test, y_pred)      

    return final_results

In [117]:
final_res = model_for_different_gamemodes()

Best Params:  {'n_estimators': 300, 'min_samples_split': 51, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 15}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Best Params:  {'n_estimators': 200, 'min_samples_split': 16, 'min_samples_leaf': 44, 'max_features': 'sqrt', 'max_depth': 50}
Best Params:  {'n_estimators': 100, 'min_samples_split': 30, 'min_samples_leaf': 30, 'max_features': 'log2', 'max_depth': 22}
Best Params:  {'n_estimators': 400, 'min_samples_split': 16, 'min_samples_leaf': 9, 'max_features': 'sqrt', 'max_depth': 57}
Best Params:  {'n_estimators': 200, 'min_samples_split': 9, 'min_samples_leaf': 23, 'max_features': 'sqrt', 'max_depth': 36}


In [118]:
pprint.pprint(final_res)

{'all_data': {'Test Set metics': {'accuracy_score': 0.5602953240360952,
                                  'f1_score': 0.5359514062880057,
                                  'precision_score': 0.5942112121767578},
              'Trainging Set metics': {'accuracy_score': 0.6331053351573187,
                                       'f1_score': 0.6108380453728854,
                                       'precision_score': 0.6580530996085001}},
 'blitz': {'Test Set metics': {'accuracy_score': 0.5597189695550351,
                               'f1_score': 0.5419790595262634,
                               'precision_score': 0.5844459980813757},
           'Trainging Set metics': {'accuracy_score': 0.6893317702227433,
                                    'f1_score': 0.6705987079518879,
                                    'precision_score': 0.7108103146056876}},
 'bullet': {'Test Set metics': {'accuracy_score': 0.5327868852459017,
                                'f1_score': 0.4631352254303073,
    

In [90]:
classifiers = {
    'Random Forest': {
        'model': RandomForestClassifier(),
        'param_dist': {
            'n_estimators': [100, 200, 300],
            'max_depth': [5, 10, 15],
            'min_samples_leaf': [1, 5, 10],
            'min_samples_split': [2, 5, 10],
            'max_features': ['sqrt', 'log2']
        }
    },
    'Logistic Regression': {
        'model': LogisticRegression(),
        'param_dist': {
            'C': [0.1, 1.0, 10.0],
            'penalty': ['l1', 'l2']
        }
    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'param_dist': {
            'n_neighbors': [3, 5, 7],
            'weights': ['uniform', 'distance']
        }
    },
    'SVM': {
        'model': SVC(),
        'param_dist': {
            'C': [0.1, 1.0, 10.0],
            'kernel': ['linear', 'rbf', 'sigmoid']
        }
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(),
        'param_dist': {
            'criterion': ['gini', 'entropy'],
            'max_depth': [5, 10, 15],
            'min_samples_leaf': [1, 5, 10],
            'min_samples_split': [2, 5, 10],
            'max_features': ['sqrt', 'log2']
        }
    },
    'Naive Bayes': {
        'model': GaussianNB(),
        'param_dist': {}
    }
}

In [147]:
def model_for_different_gamemodes_extended():
    final_results = {}
    all_dataframes = [blitz, classical, bullet, rapid, final_all]
    dataframes_names_str = ["blitz", "classical", "bullet", "rapid", "all_data"]
    classifiers = {
    'Random Forest': {
        'model': RandomForestClassifier(),
        'param_dist': {
            'n_estimators': [100, 200, 300],
            'max_depth': [5, 10, 15],
            'min_samples_leaf': [1, 5, 10],
            'min_samples_split': [2, 5, 10],
            'max_features': ['sqrt', 'log2']
        }
    },
    'Logistic Regression': {
        'model': LogisticRegression(max_iter=1000),
        'param_dist': {
            'C': [0.1, 1.0, 10.0],
            'penalty': ['l2', None]
        }
    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'param_dist': {
            'n_neighbors': [3, 5, 7],
            'weights': ['uniform', 'distance']
        }
    },
    'SVM': {
        'model': SVC(),
        'param_dist': {
            'C': [0.1, 1.0, 10.0],
            'kernel': ['linear', 'rbf', 'sigmoid']
        }
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(),
        'param_dist': {
            'criterion': ['gini', 'entropy'],
            'max_depth': [5, 10, 15],
            'min_samples_leaf': [1, 5, 10],
            'min_samples_split': [2, 5, 10],
            'max_features': ['sqrt', 'log2']
        }
    },
    'Naive Bayes': {
        'model': GaussianNB(),
        'param_dist': {}
    }
    }
    
    for dataframe, dataframe_name in zip(all_dataframes, dataframes_names_str):
        
        X = dataframe.drop("i_won", axis=1)
        y = dataframe['i_won']
        # Define the parameter grid for grid search
        
        scaler = MinMaxScaler()
        X_normalized = scaler.fit_transform(X)

        X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, random_state = 0)  
        
        final_results[dataframe_name] = {}

        
        for model_name, model_features in classifiers.items():
            
            # Initlialize a model
            model = model_features['model']
            
            param_dist = model_features['param_dist']
            

            # Create the randomized search object
            random_search = RandomizedSearchCV(model, param_distributions=param_dist, cv=5, n_iter=10, error_score='raise')

            # Perform randomized search on the training data
            random_search.fit(X_train, y_train)

            # Get the best estimator and its corresponding parameters
            best_model = random_search.best_estimator_

            # Fit the best estimator on the training data
            best_model.fit(X_train, y_train)

            # Predict on the training and test data using the best estimator
            y_pred_train = best_model.predict(X_train)
            y_pred = best_model.predict(X_test)
            
            final_results[dataframe_name][model_name] = {}

            # Calculate the evaluation metrics
            final_results[dataframe_name][model_name]["Best Estimators: "] = random_search.best_params_
            final_results[dataframe_name][model_name]["Trainging Set metics"] = {}
            final_results[dataframe_name][model_name]["Trainging Set metics"]['f1_score'] = f1_score(y_train, y_pred_train, average='weighted', zero_division=1)
            final_results[dataframe_name][model_name]["Trainging Set metics"]['precision_score'] = precision_score(y_train, y_pred_train, average='weighted', zero_division=1)
            final_results[dataframe_name][model_name]["Trainging Set metics"]['accuracy_score'] = accuracy_score(y_train, y_pred_train)

            final_results[dataframe_name][model_name]["Test Set metics"] = {}
            final_results[dataframe_name][model_name]["Test Set metics"]['f1_score'] = f1_score(y_test, y_pred, average='weighted', zero_division=1)
            final_results[dataframe_name][model_name]["Test Set metics"]['precision_score'] = precision_score(y_test, y_pred, average='weighted', zero_division=1)
            final_results[dataframe_name][model_name]["Test Set metics"]['accuracy_score'] = accuracy_score(y_test, y_pred)      
        print(model_name, " finished :)")
    return final_results

In [148]:
fin_res = {}
fin_res["bulety_pociski"] = {}
for dupa in ["cycki", "xd", "dupa", "fff"]:
    fin_res["bulety_pociski"][] = dupa

fin_res

SyntaxError: invalid syntax (2755556995.py, line 4)

In [149]:
final_res = model_for_different_gamemodes_extended()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn

Naive Bayes  finished :)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/sit

Naive Bayes  finished :)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn

...

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

Naive Bayes  finished :)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn

Naive Bayes  finished :)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [150]:
pprint.pprint(final_res)

{'all_data': {'Decision Tree': {'Best Estimators: ': {'criterion': 'gini',
                                                      'max_depth': 5,
                                                      'max_features': 'sqrt',
                                                      'min_samples_leaf': 10,
                                                      'min_samples_split': 5},
                                'Test Set metics': {'accuracy_score': 0.4946677604593929,
                                                    'f1_score': 0.47124406603235325,
                                                    'precision_score': 0.5231667960184042},
                                'Trainging Set metics': {'accuracy_score': 0.5244870041039672,
                                                         'f1_score': 0.5056056116262427,
                                                         'precision_score': 0.5416122143420976}},
              'KNN': {'Best Estimators: ': {'n_neighbors': 3,
         

In [127]:
for model_name, model_features in classifiers.items():
    print(model_name)
    print(type(model_name))

Random Forest
<class 'str'>
Logistic Regression
<class 'str'>
KNN
<class 'str'>
SVM
<class 'str'>
Decision Tree
<class 'str'>
Naive Bayes
<class 'str'>


In [123]:
for x, y in classifiers.items():
    print("X: ", x)
    print("Y: ", y)
    print("y_model: ", y['model'])
    print("y_param: ", y['param_dist'], "\n\n")

X:  Random Forest
Y:  {'model': RandomForestClassifier(), 'param_dist': {'n_estimators': [100, 200, 300], 'max_depth': [5, 10, 15], 'min_samples_leaf': [1, 5, 10], 'min_samples_split': [2, 5, 10], 'max_features': ['sqrt', 'log2']}}
y_model:  RandomForestClassifier()
y_param:  {'n_estimators': [100, 200, 300], 'max_depth': [5, 10, 15], 'min_samples_leaf': [1, 5, 10], 'min_samples_split': [2, 5, 10], 'max_features': ['sqrt', 'log2']} 


dict_keys(['model', 'param_dist'])
X:  Logistic Regression
Y:  {'model': LogisticRegression(), 'param_dist': {'C': [0.1, 1.0, 10.0], 'penalty': ['l1', 'l2']}}
y_model:  LogisticRegression()
y_param:  {'C': [0.1, 1.0, 10.0], 'penalty': ['l1', 'l2']} 


dict_keys(['model', 'param_dist'])
X:  KNN
Y:  {'model': KNeighborsClassifier(), 'param_dist': {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']}}
y_model:  KNeighborsClassifier()
y_param:  {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']} 


dict_keys(['model', 'param_dist'])
X:  SV

In [151]:
type(fin_res)

dict

In [160]:
wyniki_bezpieczne = final_res.copy()

In [161]:
final_res.keys()

dict_keys(['blitz', 'classical', 'bullet', 'rapid', 'all_data'])

In [162]:
wyniki = pd.DataFrame.from_dict(final_res)

In [165]:
wyniki.to_excel("wyniczki_turbo_funkcji.xlsx")

ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
w